In [1]:
# This notebook applies the GXBoost model to a years worth a cliamte data. 

In [2]:
import pickle
import glob
from tqdm.auto import tqdm
import os
from datetime import datetime
import dateutil.relativedelta
import json
from pathlib import Path
from threading import Thread
import warnings

import geopandas as gpd
import pandas as pd
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from multiprocessing.pool import ThreadPool


In [3]:
model_name = 'dam_forecast_11-10-22.pkl'

In [4]:
# read in vars
%store -r dam_forcast_working_dir
%store -r min_year
%store -r max_year
%store -r climate_types
%store -r point_data_input_path
%store -r dam_forcast_working_dir
%store -r time_step_folder
# set up some folders
output_folder = os.path.join(dam_forcast_working_dir,'monthly output')
Path(output_folder).mkdir(exist_ok=True)
# location of model file
model_folder = os.path.join(os.getcwd(),'data')
model_path = os.path.join(model_folder,model_name)
'found model?',os.path.isfile(model_path)

('found model?', True)

In [5]:
# load in data from meta file
with open(model_path.replace('.pkl','.json'), 'r') as fp:
    meta_dict = json.load(fp)

In [6]:
# load up model
xgb_reg_models = pickle.load(open(model_path, "rb"))

In [7]:
# get a refence to each time step
time_steps = glob.glob(time_step_folder+'/*.pkl')
len(time_steps)

144

In [8]:
# open vector data
dam_points = gpd.read_file(point_data_input_path)
dam_points.head()
# reproject to WGS 84 if not already
if dam_points.crs.to_epsg() != 4326:
    print('Reprojecting')
    dam_points = dam_points.to_crs("EPSG:4326")

Reprojecting


In [9]:
# this func takes in input date and return 13 months worth climate data for each point
def build_inf_df(start_date):
#     convert string date to datetime
    start_date_dt = datetime.strptime(start_date, '%Y%m')
    point_sample_df_list = []
#     loop over each calimte and relative month
    for climate_type in climate_types:
            for month_shift in range(0,13):
#                 get actual date from relative month
                shifted_date_dt = start_date_dt - dateutil.relativedelta.relativedelta(months=month_shift)
#                 build export path
                month_str = str(shifted_date_dt.month).zfill(2)
                file_name = f'{shifted_date_dt.year}{month_str}_{climate_type}.pkl'
                relative_date_name = f'{climate_type}_{month_shift}_months before'
                time_step_file_path = os.path.join(time_step_folder,file_name)
#                 open file and name col name relative
                try:
                    point_sample_df = pd.read_pickle(time_step_file_path)

                    point_sample_df.rename(columns={point_sample_df.columns[0]: relative_date_name}, inplace=True)

                    point_sample_df_list.append(point_sample_df)
                except Exception as e:
                    print(e)
#     combine all data
    all_time_steps = pd.concat(point_sample_df_list,axis=1)
    
    all_time_steps_with_point_data = pd.concat([dam_points['dam_area'],all_time_steps],axis=1)
#     make sure all data is in correct order
    all_time_steps_with_point_data = all_time_steps_with_point_data.reindex(meta_dict['col_names'], axis=1)

    return all_time_steps_with_point_data        

In [10]:
# run a test extraction
test_df = build_inf_df('202101')
test_df.head()

,dam_area,rain_0_months before,rain_10_months before,rain_11_months before,rain_12_months before,rain_1_months before,rain_2_months before,rain_3_months before,rain_4_months before,rain_5_months before,...,tavg_12_months before,tavg_1_months before,tavg_2_months before,tavg_3_months before,tavg_4_months before,tavg_5_months before,tavg_6_months before,tavg_7_months before,tavg_8_months before,tavg_9_months before
0,1782.758036,91.389145,102.111046,15.942660,34.318153,41.518482,29.074463,66.378281,41.120415,80.803795,...,24.253502,20.431652,20.566591,15.726265,12.332435,8.657789,8.072989,8.438752,10.184371,14.459160
1,1917.350912,91.599861,60.374527,86.209282,103.682495,92.023117,43.234932,137.324905,85.224243,133.838364,...,18.292000,15.620107,16.618374,13.130466,11.563720,9.255119,8.712623,9.055582,10.510744,13.301476
2,2300.790143,94.956673,101.596077,55.016685,65.366745,87.820847,52.842342,129.010971,107.157036,113.654251,...,15.991950,13.331125,13.894073,10.666141,9.386144,7.773449,7.490423,8.403605,9.665147,11.672211
3,403.480805,25.214075,10.155920,46.595879,18.252161,24.598694,21.089201,67.211670,64.161491,58.338249,...,20.686340,17.684000,19.030613,13.551085,12.200335,9.097849,8.387416,8.973354,10.934418,14.367694
4,1774.881116,48.494244,111.083740,99.955391,46.732124,128.312881,73.006340,86.361069,71.220161,54.039902,...,24.156582,18.429838,17.916143,14.671591,11.413791,7.220846,7.222924,7.851370,9.137348,13.591210


In [11]:
# the above df should have these cols
meta_dict['col_names']

['dam_area',
 'rain_0_months before',
 'rain_10_months before',
 'rain_11_months before',
 'rain_12_months before',
 'rain_1_months before',
 'rain_2_months before',
 'rain_3_months before',
 'rain_4_months before',
 'rain_5_months before',
 'rain_6_months before',
 'rain_7_months before',
 'rain_8_months before',
 'rain_9_months before',
 'tavg_0_months before',
 'tavg_10_months before',
 'tavg_11_months before',
 'tavg_12_months before',
 'tavg_1_months before',
 'tavg_2_months before',
 'tavg_3_months before',
 'tavg_4_months before',
 'tavg_5_months before',
 'tavg_6_months before',
 'tavg_7_months before',
 'tavg_8_months before',
 'tavg_9_months before']

In [12]:
# make a list of the start times
time_steps = []
for year in range(min_year,max_year+1):
#     loop over each month
    for month in range(1,13):
        month = str(month).zfill(2)
        time_steps.append(f'{year}{month}')
len(time_steps)

60

In [13]:
# func to run the model when given a start time
def run_model(time_step):
#     build inf df
    ts_df = build_inf_df(time_step)
#     loop over each model, and combine preds
    preds_df = pd.DataFrame()
    for model in xgb_reg_models:
        preds = model['model'].predict(ts_df)
        preds = preds*(preds>0)
        preds_df[model['model_number']] = preds
        
    preds_mean = preds_df.mean(axis=1).to_list()
    
#     make df with just files names to index to
    limited_df = pd.DataFrame(dam_points['file_name'].copy())
#     place the preds into df with time a col heading
    limited_df[time_step] = preds_mean
#     build export path
    file_name = f'{time_step}.pkl'
    pkl_out_path = os.path.join(output_folder,file_name)
#     export to disk
    limited_df.to_pickle(pkl_out_path)
    
    return pkl_out_path

In [14]:
with ThreadPool(2) as p:
    monthly_ouputs = list(tqdm(p.imap(run_model,time_steps),total=len(time_steps)))

  0%|          | 0/60 [00:00<?, ?it/s]

In [15]:
monthly_ouputs = glob.glob(output_folder+'/*.pkl')

In [16]:
# get a list of all prediction files
len(monthly_ouputs)

60

In [17]:
dam_points.head()

,file_name,dam_area,water_area,SRC_DATE,file_name_pred,class,class_name,geometry
0,arcgisonline_1286093.tif,1782.758036,1231.106859,20170126.0,arcgisonline_1286093_pred_and_score.tif,0,dam and water,POINT (146.65881 -36.09349)
1,arcgisonline_1337387.tif,1917.350912,1042.750433,20151214.0,arcgisonline_1337387_pred_and_score.tif,0,dam and water,POINT (145.80580 -38.45117)
2,arcgisonline_1702518.tif,2300.790143,698.207560,20141023.0,arcgisonline_1702518_pred_and_score.tif,0,dam and water,POINT (145.69447 -41.09943)
3,arcgisonline_1274916.tif,403.480805,90.273377,20160112.0,arcgisonline_1274916_pred_and_score.tif,0,dam and water,POINT (141.22659 -36.64662)
4,arcgisonline_704977.tif,1774.881116,749.984968,20160502.0,arcgisonline_704977_pred_and_score.tif,0,dam and water,POINT (149.83683 -32.87746)


In [18]:
# make empty df to load preds into, we need to keep this separate from the base data so we can sort it by date
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
preds_df = pd.DataFrame()
# loop over each pred csv
for pkl in tqdm(monthly_ouputs):
#     get the file name
    file_name = os.path.basename(pkl)
#     get the date from the name
    date = file_name.replace('.pkl','')
#     open file
    current_df = pd.read_pickle(pkl)
#     extract the preds col
    preds = current_df[date].to_list()
#     name date a proper python date
    date_fixed = datetime.strptime(date, '%Y%m')
#     load preds into main df
    preds_df[date_fixed] = preds
# sort the cols by date
preds_df = preds_df[sorted(preds_df.columns)]

  0%|          | 0/60 [00:00<?, ?it/s]

In [19]:
# set any negative values to 0
preds_df[preds_df < 0] = 0
preds_df.head()

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-09-01,2017-10-01,...,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01
0,952.322388,927.305481,711.968018,540.521729,705.663635,666.602112,645.062866,738.790588,501.365143,632.762207,...,770.659729,616.691101,701.735229,792.579224,699.856812,933.390442,1071.022705,1000.248169,834.537598,878.112427
1,1046.007202,1116.355103,1010.173523,1101.135864,933.407104,1118.337646,1062.534790,1049.397705,1185.483398,1139.094116,...,1008.033386,1052.309204,963.690247,1125.431641,1116.149536,1119.865356,1151.517700,1154.919678,1204.607300,1100.688110
2,1151.970703,1264.447510,1219.902466,931.187500,868.624023,1004.369629,1159.568726,1312.796753,1225.346436,1288.260254,...,1235.473755,1303.700928,1429.981323,1231.614990,1401.031372,1277.482666,1271.489014,1336.824219,1361.228516,1422.942627
3,124.076698,120.006508,98.038406,109.651085,140.576416,132.834808,137.317871,139.984314,182.983948,162.431824,...,89.383865,80.642578,72.700150,105.873962,127.059464,150.670975,147.462799,152.620621,152.511322,113.881287
4,993.129395,803.295654,784.939453,689.728088,916.158386,734.790710,410.953461,597.852173,354.739441,122.664467,...,1015.452820,604.552917,851.009155,726.915527,837.709473,961.423218,953.377441,995.160034,837.608276,983.080872


In [20]:
# join the input dam points data with the preds
preds_with_meta = pd.concat([dam_points, preds_df], axis=1)
preds_with_meta.head()

,file_name,dam_area,water_area,SRC_DATE,file_name_pred,class,class_name,geometry,2017-01-01 00:00:00,2017-02-01 00:00:00,...,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758036,1231.106859,20170126.0,arcgisonline_1286093_pred_and_score.tif,0,dam and water,POINT (146.65881 -36.09349),952.322388,927.305481,...,770.659729,616.691101,701.735229,792.579224,699.856812,933.390442,1071.022705,1000.248169,834.537598,878.112427
1,arcgisonline_1337387.tif,1917.350912,1042.750433,20151214.0,arcgisonline_1337387_pred_and_score.tif,0,dam and water,POINT (145.80580 -38.45117),1046.007202,1116.355103,...,1008.033386,1052.309204,963.690247,1125.431641,1116.149536,1119.865356,1151.517700,1154.919678,1204.607300,1100.688110
2,arcgisonline_1702518.tif,2300.790143,698.207560,20141023.0,arcgisonline_1702518_pred_and_score.tif,0,dam and water,POINT (145.69447 -41.09943),1151.970703,1264.447510,...,1235.473755,1303.700928,1429.981323,1231.614990,1401.031372,1277.482666,1271.489014,1336.824219,1361.228516,1422.942627
3,arcgisonline_1274916.tif,403.480805,90.273377,20160112.0,arcgisonline_1274916_pred_and_score.tif,0,dam and water,POINT (141.22659 -36.64662),124.076698,120.006508,...,89.383865,80.642578,72.700150,105.873962,127.059464,150.670975,147.462799,152.620621,152.511322,113.881287
4,arcgisonline_704977.tif,1774.881116,749.984968,20160502.0,arcgisonline_704977_pred_and_score.tif,0,dam and water,POINT (149.83683 -32.87746),993.129395,803.295654,...,1015.452820,604.552917,851.009155,726.915527,837.709473,961.423218,953.377441,995.160034,837.608276,983.080872


In [21]:
# convert col headings to strings to improve file compatability
str_names_dict = {}
for name in preds_with_meta.columns.to_list():
    str_names_dict[name] = str(name)
    
preds_with_meta = preds_with_meta.rename(index=str, columns=str_names_dict)
preds_with_meta.head()

,file_name,dam_area,water_area,SRC_DATE,file_name_pred,class,class_name,geometry,2017-01-01 00:00:00,2017-02-01 00:00:00,...,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,2021-09-01 00:00:00,2021-10-01 00:00:00,2021-11-01 00:00:00,2021-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758036,1231.106859,20170126.0,arcgisonline_1286093_pred_and_score.tif,0,dam and water,POINT (146.65881 -36.09349),952.322388,927.305481,...,770.659729,616.691101,701.735229,792.579224,699.856812,933.390442,1071.022705,1000.248169,834.537598,878.112427
1,arcgisonline_1337387.tif,1917.350912,1042.750433,20151214.0,arcgisonline_1337387_pred_and_score.tif,0,dam and water,POINT (145.80580 -38.45117),1046.007202,1116.355103,...,1008.033386,1052.309204,963.690247,1125.431641,1116.149536,1119.865356,1151.517700,1154.919678,1204.607300,1100.688110
2,arcgisonline_1702518.tif,2300.790143,698.207560,20141023.0,arcgisonline_1702518_pred_and_score.tif,0,dam and water,POINT (145.69447 -41.09943),1151.970703,1264.447510,...,1235.473755,1303.700928,1429.981323,1231.614990,1401.031372,1277.482666,1271.489014,1336.824219,1361.228516,1422.942627
3,arcgisonline_1274916.tif,403.480805,90.273377,20160112.0,arcgisonline_1274916_pred_and_score.tif,0,dam and water,POINT (141.22659 -36.64662),124.076698,120.006508,...,89.383865,80.642578,72.700150,105.873962,127.059464,150.670975,147.462799,152.620621,152.511322,113.881287
4,arcgisonline_704977.tif,1774.881116,749.984968,20160502.0,arcgisonline_704977_pred_and_score.tif,0,dam and water,POINT (149.83683 -32.87746),993.129395,803.295654,...,1015.452820,604.552917,851.009155,726.915527,837.709473,961.423218,953.377441,995.160034,837.608276,983.080872


In [22]:
# make csv export path
final_export = os.path.join(dam_forcast_working_dir,'Dam forecast preds v7.csv')
final_export

'/home/nick/Documents/Work code/Weather-to-water/working/Dam forecast preds v7.csv'

In [23]:
# make gpkg export path
gpkg_file_name = final_export.replace('.csv','.gpkg')
gpkg_file_name

'/home/nick/Documents/Work code/Weather-to-water/working/Dam forecast preds v7.gpkg'

In [24]:
pkl_file_name = final_export.replace('.csv','.pkl')
pkl_file_name


'/home/nick/Documents/Work code/Weather-to-water/working/Dam forecast preds v7.pkl'

In [25]:
preds_with_meta.to_pickle(pkl_file_name)

In [26]:
# export files
preds_with_meta.to_csv(final_export)
final_export

'/home/nick/Documents/Work code/Weather-to-water/working/Dam forecast preds v7.csv'

In [27]:
preds_with_meta.to_file(gpkg_file_name,driver='GPKG')
gpkg_file_name

'/home/nick/Documents/Work code/Weather-to-water/working/Dam forecast preds v7.gpkg'